# Loading Set

In [1]:
from keras import losses, metrics
from keras.models import Sequential
from keras.models import Model, load_model
from keras.optimizers import Adam as Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D
from keras.layers import Input, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, sys, pickle
import pandas as pd
import random

In [2]:
import pandas as pd

print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
#input_path = '../dataset/preprocess4/input2/PPG_100Hz/'
input_path = '../dataset/preprocess4/input2/'
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
y_train = np.load(input_path+'y_train.npz')['arr_0']

gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']


x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
x_val_pacu = np.load(input_path+'x_val_pacu.npz', allow_pickle=True)['arr_0']
y_val_pacu =  np.load(input_path+'y_val_pacu.npz')['arr_0']




x_val_pacu = np.transpose(x_val_pacu, [0,2,1])
x_val_pacu[:,:,0] = pd.DataFrame(x_val_pacu[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
x_val_pacu[:,:,1] = pd.DataFrame(x_val_pacu[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values

x_test_pacu = np.transpose(x_test_pacu, [0,2,1])
x_test_pacu[:,:,0] = pd.DataFrame(x_test_pacu[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
x_test_pacu[:,:,1] = pd.DataFrame(x_test_pacu[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values


# binary classification
y_train_bin = y_train >= 4
y_test_bin = y_test_pacu>=4
y_val_bin = y_val_pacu>=4


print('x_train shape:', x_train.shape)
print('x_test.shape:', x_test_pacu.shape)
print('x_val.shape:', x_val_pacu.shape)

print('done', flush=True)


loading train...x_train shape: (74636, 5000, 2)
x_test.shape: (7497, 5000, 2)
x_val.shape: (6724, 5000, 2)
done


## pacu set for val, test set & sample weights(3 class)

In [3]:
# train: age+gender
age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']

agender_train = np.array([[age_train[i], gender_train[i]] for i in range(len(age_train))])


# val: age+gender
age_val_pacu = np.load(input_path+'age_val_pacu.npz', allow_pickle=True)['arr_0']
gender_val_pacu = np.load(input_path+'gender_val_pacu.npz', allow_pickle=True)['arr_0']

agender_val_pacu = np.array([[age_val_pacu[i], gender_val_pacu[i]] for i in range(len(age_val_pacu))])


# test: age+gender
age_test_pacu = np.load(input_path+'age_test_pacu.npz', allow_pickle=True)['arr_0']
gender_test_pacu = np.load(input_path+'gender_test_pacu.npz', allow_pickle=True)['arr_0']

agender_test_pacu = np.array([[age_test_pacu[i], gender_test_pacu[i]] for i in range(len(age_test_pacu))])


# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train),))

train_w_samp3[y_train<4]= len(y_train)/np.sum(y_train<4)
train_w_samp3[(4<=y_train)&(y_train<7)]= len(y_train)/np.sum((4<=y_train)&(y_train<7))
train_w_samp3[7<=y_train]= len(y_train)/np.sum(7<=y_train)

print('train set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_train)/np.sum(y_train<4),len(y_train)/np.sum((4<=y_train)&(y_train<7)),len(y_train)/np.sum(7<=y_train)))


# val sample weight
val_w_samp3 = np.ones(shape=(len(y_val_pacu),))

val_w_samp3[y_val_pacu<4]= len(y_val_pacu)/np.sum(y_val_pacu<4)
val_w_samp3[(4<=y_val_pacu)&(y_val_pacu<7)]= len(y_val_pacu)/np.sum((4<=y_val_pacu)&(y_val_pacu<7))
val_w_samp3[7<=y_val_pacu]= len(y_val_pacu)/np.sum(7<=y_val_pacu)

print('val_pacu set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_val_pacu)/np.sum(y_val_pacu<4),len(y_val_pacu)/np.sum((4<=y_val_pacu)&(y_val_pacu<7)),len(y_val_pacu)/np.sum(7<=y_val_pacu)))


# test sample weight
test_w_samp3 = np.ones(shape=(len(y_test_pacu),))

test_w_samp3[y_test_pacu<4]= len(y_test_pacu)/np.sum(y_test_pacu<4)
test_w_samp3[(4<=y_test_pacu)&(y_test_pacu<7)]= len(y_test_pacu)/np.sum((4<=y_test_pacu)&(y_test_pacu<7))
test_w_samp3[7<=y_test_pacu]= len(y_test_pacu)/np.sum(7<=y_test_pacu)

print('test_pacu set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}'
      .format(len(y_test_pacu)/np.sum(y_test_pacu<4),len(y_test_pacu)/np.sum((4<=y_test_pacu)&(y_test_pacu<7)),len(y_test_pacu)/np.sum(7<=y_test_pacu)))

train set
sample weight for class 1: 2.84, class 2: 2.07, class 3: 6.08

val_pacu set
sample weight for class 1: 3.90, class 2: 1.77, class 3: 5.61

test_pacu set
sample weight for class 1: 3.64, class 2: 1.85, class 3: 5.42


## pacu set for train, val, test set

In [5]:
x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
y_train_pacu =  np.load(input_path+'y_train_pacu.npz')['arr_0']

x_train_pacu = np.transpose(x_train_pacu, [0,2,1])
x_train_pacu[:,:,0] = pd.DataFrame(x_train_pacu[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
x_train_pacu[:,:,1] = pd.DataFrame(x_train_pacu[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values

print('x_train shape:', x_train_pacu.shape)


# train_pacu: age+gender
age_train_pacu = np.load(input_path+'age_train_pacu.npz', allow_pickle=True)['arr_0']
gender_train_pacu = np.load(input_path+'gender_train_pacu.npz', allow_pickle=True)['arr_0']

agender_train_pacu = np.array([[age_train_pacu[i], gender_train_pacu[i]] for i in range(len(age_train_pacu))])

# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train_pacu),))

train_w_samp3[y_train_pacu<4]= len(y_train_pacu)/np.sum(y_train_pacu<4)
train_w_samp3[(4<=y_train_pacu)&(y_train_pacu<7)]= len(y_train_pacu)/np.sum((4<=y_train_pacu)&(y_train_pacu<7))
train_w_samp3[7<=y_train_pacu]= len(y_train_pacu)/np.sum(7<=y_train_pacu)

print('train_pacu set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_train_pacu)/np.sum(y_train_pacu<4),len(y_train_pacu)/np.sum((4<=y_train_pacu)&(y_train_pacu<7)),len(y_train_pacu)/np.sum(7<=y_train_pacu)))



x_train shape: (63256, 5000, 2)
train_pacu set
sample weight for class 1: 4.23, class 2: 1.75, class 3: 5.15



# Settings

In [6]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 4 Logical GPUs


In [18]:
# folder
nfold = 1  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 100
rootdir = "ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train"

predirs = []
for root, dirs, files in os.walk(rootdir):  # 하위 대상들을 recursive 하게 긁어옴
    for filename in dirs:
        predirs.append(filename)

if not os.path.exists(rootdir):
    os.mkdir(rootdir)


# test_settings
test_settings_1, test_settings_2, test_settings_3, test_settings_4 = [], [], [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.002

# hyperparamters pool
num_opts = [32, 64, 128, 256] # num of filters(kernel)
stride_opts = [1,1,1,1,1,2,2,2,2,3]
kernel_opts = [3,3,3,3,3,5,5,5,5,7] # kernel size
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [16, 32, 64, 128]
globalpool_opts = ['max','ave']
BATCH_SIZE = [512, 1024]


print('start making test settings...', end='', flush=True)
# test settings
for num_l1 in num_opts:
    for num_l2 in num_opts:
        for num_l3 in num_opts:
            for num_l4 in num_opts:
                for kernel_l1 in kernel_opts:
                    for kernel_l2 in kernel_opts:
                        for kernel_l3 in kernel_opts:
                            for kernel_l4 in kernel_opts:
                                test_settings_1.append([num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4])

for dense1 in dense_opts:
    for dropout1 in dropout_opts:
        for dropout_fc in dropout_opts:
            for globalpool_opt in globalpool_opts:
                for batch_size in BATCH_SIZE:
                    for conv_double in [True, False]:
                        test_settings_2.append([dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double])                                   

for stride_l1 in stride_opts:
    for stride_l2 in stride_opts:
        for stride_l3 in stride_opts:
            for stride_l4 in stride_opts:
                for stride_l5 in stride_opts:
                    for num_l5 in num_opts:
                        for kernel_l5 in kernel_opts:
                            test_settings_3.append([stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5])
      
for dense2 in dense_opts:
    for dropout2 in dropout_opts:
        test_settings_4.append([dense2, dropout2])
    
print('done')

start making test settings...done


# RandomSearch

## binary

### val, train pacu set

In [ ]:
# ECG
train_w_samp = train_w_samp3
val_w_samp = val_w_samp3
test_w_samp = test_w_samp3


x_train = x_train[:,:,1:2]
x_val_pacu = x_val_pacu[:,:,1:2]
x_test_pacu = x_test_pacu[:,:,1:2]


x_trains = [x_train, agender_train]
x_vals = [x_val_pacu, agender_val_pacu]
x_tests = [x_test_pacu, agender_test_pacu]

In [8]:
# ECG - all pacu
train_w_samp = train_w_samp3
val_w_samp = val_w_samp3
test_w_samp = test_w_samp3


x_train_pacu = x_train_pacu[:,:,1:2]
x_val_pacu = x_val_pacu[:,:,1:2]
x_test_pacu = x_test_pacu[:,:,1:2]

x_train = x_train_pacu
agender_train = agender_train_pacu


y_train_bin = y_train_pacu>=4
y_test_bin = y_test_pacu>=4
y_val_bin = y_val_pacu>=4


x_trains = [x_train_pacu, agender_train_pacu]
x_vals = [x_val_pacu, agender_val_pacu]
x_tests = [x_test_pacu, agender_test_pacu]

print(x_train_pacu.shape, len(agender_train_pacu), len(train_w_samp3))
print(x_val_pacu.shape, len(agender_val_pacu), len(val_w_samp3))
print(x_test_pacu.shape, len(agender_test_pacu), len(test_w_samp3))

(63256, 5000, 1) 63256
(6724, 5000, 1) 6724
(7497, 5000, 1) 7497


In [20]:
from keras import metrics
# random search for hyperparameter
ntrial = 400
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    test_setting_3 = random.choice(test_settings_3)
    test_setting_4 = random.choice(test_settings_4)    
    
        
    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2
    stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5 = test_setting_3
    dense2, dropout2 = test_setting_4
    
    
    # total conv layers of the model
    n_conv = random.choice([2,3])
    
    if n_conv==2:
        num_l3,kernel_l3,stride_l3 = 0,0,0
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
    
    if n_conv==3:
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
        
    if n_conv==4:
        num_l5,kernel_l5,stride_l5 = 0,0,0  
        
    
    n_dense = random.choice([0,1,2])
    
    if n_dense==0:
        dense1, dropout1 = 0,0
        dense2, dropout2 = 0,0
    
    if n_dense==1:
        dense2, dropout2 = 0,0
        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={},c1={},c2={},c3={},filt1={},filt2={},filt3={},str1={},str2={},str3={},conv_double={},globalpool={},dropout={},dnodes={},dropout={},dnodes={},dropout={}'.format(batch_size, num_l1, num_l2, num_l3,kernel_l1, kernel_l2, kernel_l3, stride_l1,stride_l2,stride_l3,conv_double, globalpool_opt, dropout_fc, dense1, dropout1, dense2, dropout2)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
    with strategy.scope():
        # build a model
        inp_fnn = Input(shape=(agender_train.shape[1],))
        inp_cnn = Input(shape=(x_train.shape[1], x_train.shape[2]))
    
        out_fnn = inp_fnn
        out_fnn = Activation('sigmoid') (out_fnn)
        
        out_cnn = inp_cnn
        act='relu'
        
        
        # c1 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)


        # c2 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        
        
        # c3 layer
        if n_conv>2:
            if conv_double:
                out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        
        
        # c4 layer
        if n_conv>3:
            if num_l3 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
            
                    


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            out_cnn = GlobalMaxPool1D() (out_cnn)
        elif globalpool_opt == 'ave':
            out_cnn = GlobalAveragePooling1D() (out_cnn)    
        out_cnn = Dropout(dropout_fc) (out_cnn)
        
        
        out = concatenate([out_fnn, out_cnn])
            
        if n_dense>=1:
            out = Dense(dense1, activation='tanh') (out)
            out = Dropout(dropout1) (out)
            
        if n_dense==2:
            out = Dense(dense2, activation='tanh') (out)
            out = Dropout(dropout2) (out)
            
        out = Dense(1, activation='sigmoid') (out)
        
        
        model = Model(inputs=[inp_cnn, inp_fnn], outputs=[out])


        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_trains, y_train_bin, sample_weight=train_w_samp, validation_data=(x_vals, y_val_bin, val_w_samp), epochs=50, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except Exception as e:
            print(e)
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_tests).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))    
    
    
    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    
    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


random search 0/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/50
61/62 [============================>.] - ETA: 0s - loss: 2.0925 - acc: 0.6826 - auc_1: 0.5057
Epoch 00001: val_loss improved from inf to 1.90470, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=1024,c1=64,c2=32,c3=0,filt1=3,filt2=5,filt3=0,str1=2,str2=2,str3=0,conv_double=False,globalpool=max,dropout=0.2,dnodes=128,dropout=0.1,dnodes=16,dropout=0.2/weights.hdf5
62/62 [==============================] - 17s 276ms/step - loss: 2.0921 - acc: 0.6827 - auc_1: 0.5059 - val_loss: 1.9047 - val_acc: 0.7638 - val_auc_1: 0.5225
Epoch 2/50
61/62 [============================>.] - ETA: 0s - loss: 2.0129 - acc: 0.7089 - auc_1: 0.5109
Epoch 00002: val_loss did not improve from 1.90470
62/62 [==============================] - 4s 62ms/step - loss: 2.0133 - acc: 0.7

124/124 [==============================] - ETA: 0s - loss: 2.0354 - acc: 0.7055 - auc: 0.4989
Epoch 00001: val_loss improved from inf to 1.91730, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=512,c1=128,c2=256,c3=0,filt1=3,filt2=3,filt3=0,str1=3,str2=2,str3=0,conv_double=True,globalpool=max,dropout=0.2,dnodes=64,dropout=0.1,dnodes=32,dropout=0.3/weights.hdf5
124/124 [==============================] - 19s 155ms/step - loss: 2.0354 - acc: 0.7055 - auc: 0.4989 - val_loss: 1.9173 - val_acc: 0.7638 - val_auc: 0.5072
Epoch 2/50
123/124 [============================>.] - ETA: 0s - loss: 1.9471 - acc: 0.7504 - auc: 0.5026
Epoch 00002: val_loss improved from 1.91730 to 1.90863, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=512,c1=128,c2=256,c3=0,filt1=3,filt2=3,filt3=0,str1=3,str2=2,str3=0,conv_double=True,globalpool=max,dropout=0.2,dnodes=64,dropout=0.1,dnodes=32,dr

Epoch 11/50
124/124 [==============================] - ETA: 0s - loss: 1.7851 - acc: 0.7440 - auc: 0.6556
Epoch 00011: val_loss did not improve from 1.76596
124/124 [==============================] - 4s 34ms/step - loss: 1.7851 - acc: 0.7440 - auc: 0.6556 - val_loss: 1.8302 - val_acc: 0.6537 - val_auc: 0.6808
Epoch 12/50
124/124 [==============================] - ETA: 0s - loss: 1.7822 - acc: 0.7399 - auc: 0.6565
Epoch 00012: val_loss did not improve from 1.76596
124/124 [==============================] - 4s 34ms/step - loss: 1.7822 - acc: 0.7399 - auc: 0.6565 - val_loss: 1.9008 - val_acc: 0.7653 - val_auc: 0.6832
Epoch 13/50
124/124 [==============================] - ETA: 0s - loss: 1.7687 - acc: 0.7431 - auc: 0.6650
Epoch 00013: val_loss did not improve from 1.76596
124/124 [==============================] - 4s 34ms/step - loss: 1.7687 - acc: 0.7431 - auc: 0.6650 - val_loss: 1.8010 - val_acc: 0.6623 - val_auc: 0.6868
random search 6/400
INFO:tensorflow:Using MirroredStrategy with dev

62/62 [==============================] - ETA: 0s - loss: 1.7277 - acc: 0.7506 - auc: 0.6826
Epoch 00013: val_loss did not improve from 1.73908
62/62 [==============================] - 14s 228ms/step - loss: 1.7277 - acc: 0.7506 - auc: 0.6826 - val_loss: 1.7662 - val_acc: 0.7657 - val_auc: 0.6959
Epoch 14/50
62/62 [==============================] - ETA: 0s - loss: 1.7133 - acc: 0.7517 - auc: 0.6905
Epoch 00014: val_loss improved from 1.73908 to 1.73207, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=1024,c1=256,c2=256,c3=0,filt1=5,filt2=5,filt3=0,str1=3,str2=1,str3=0,conv_double=True,globalpool=ave,dropout=0.2,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
62/62 [==============================] - 14s 230ms/step - loss: 1.7133 - acc: 0.7517 - auc: 0.6905 - val_loss: 1.7321 - val_acc: 0.7687 - val_auc: 0.6994
Epoch 15/50
62/62 [==============================] - ETA: 0s - loss: 1.6920 - acc: 0.7533 - auc: 0.7015
Epoch 0001

Epoch 7/50
123/124 [============================>.] - ETA: 0s - loss: 1.9036 - acc: 0.7606 - auc: 0.5287
Epoch 00007: val_loss did not improve from 1.89602
124/124 [==============================] - 5s 43ms/step - loss: 1.9033 - acc: 0.7607 - auc: 0.5289 - val_loss: 1.9068 - val_acc: 0.7638 - val_auc: 0.5362
Epoch 8/50
123/124 [============================>.] - ETA: 0s - loss: 1.9033 - acc: 0.7626 - auc: 0.5309
Epoch 00008: val_loss improved from 1.89602 to 1.89295, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=512,c1=32,c2=128,c3=0,filt1=3,filt2=5,filt3=0,str1=2,str2=1,str3=0,conv_double=False,globalpool=max,dropout=0.3,dnodes=32,dropout=0,dnodes=128,dropout=0.1/weights.hdf5
124/124 [==============================] - 5s 43ms/step - loss: 1.9040 - acc: 0.7625 - auc: 0.5309 - val_loss: 1.8930 - val_acc: 0.7645 - val_auc: 0.5468
Epoch 9/50
123/124 [============================>.] - ETA: 0s - loss: 1.9030 - acc: 0.7606 - auc

...

Epoch 12/50
62/62 [==============================] - ETA: 0s - loss: 1.5708 - acc: 0.7703 - auc: 0.7568
Epoch 00012: val_loss improved from 1.65994 to 1.65854, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=1024,c1=128,c2=256,c3=32,filt1=3,filt2=3,filt3=3,str1=1,str2=1,str3=1,conv_double=True,globalpool=ave,dropout=0,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
62/62 [==============================] - 38s 615ms/step - loss: 1.5708 - acc: 0.7703 - auc: 0.7568 - val_loss: 1.6585 - val_acc: 0.7701 - val_auc: 0.7435
Epoch 13/50
62/62 [==============================] - ETA: 0s - loss: 1.5241 - acc: 0.7801 - auc: 0.7746
Epoch 00013: val_loss did not improve from 1.65854
62/62 [==============================] - 38s 613ms/step - loss: 1.5241 - acc: 0.7801 - auc: 0.7746 - val_loss: 1.8280 - val_acc: 0.7750 - val_auc: 0.7563
Epoch 14/50
62/62 [==============================] - ETA: 0s - loss: 1.4735 - acc: 0.7880 - auc: 0.7935

Epoch 28/50
62/62 [==============================] - ETA: 0s - loss: 0.4658 - acc: 0.9541 - auc: 0.9863
Epoch 00028: val_loss improved from 0.58783 to 0.53710, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=1024,c1=128,c2=256,c3=32,filt1=3,filt2=3,filt3=3,str1=1,str2=1,str3=1,conv_double=True,globalpool=ave,dropout=0,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
62/62 [==============================] - 38s 615ms/step - loss: 0.4658 - acc: 0.9541 - auc: 0.9863 - val_loss: 0.5371 - val_acc: 0.9128 - val_auc: 0.9840
Epoch 29/50
62/62 [==============================] - ETA: 0s - loss: 0.4300 - acc: 0.9578 - auc: 0.9881
Epoch 00029: val_loss improved from 0.53710 to 0.38532, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=1024,c1=128,c2=256,c3=32,filt1=3,filt2=3,filt3=3,str1=1,str2=1,str3=1,conv_double=True,globalpool=ave,dropout=0,dnodes=0,dropout=0,dnodes=0,d

Epoch 2/50
124/124 [==============================] - ETA: 0s - loss: 1.8697 - acc: 0.7583 - auc: 0.5783
Epoch 00002: val_loss improved from 2.73470 to 2.48862, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=512,c1=32,c2=32,c3=0,filt1=3,filt2=7,filt3=0,str1=2,str2=2,str3=0,conv_double=False,globalpool=ave,dropout=0,dnodes=16,dropout=0.3,dnodes=0,dropout=0/weights.hdf5
124/124 [==============================] - 4s 29ms/step - loss: 1.8697 - acc: 0.7583 - auc: 0.5783 - val_loss: 2.4886 - val_acc: 0.2371 - val_auc: 0.5807
Epoch 3/50
124/124 [==============================] - ETA: 0s - loss: 1.8519 - acc: 0.7552 - auc: 0.5981
Epoch 00003: val_loss improved from 2.48862 to 2.00283, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=512,c1=32,c2=32,c3=0,filt1=3,filt2=7,filt3=0,str1=2,str2=2,str3=0,conv_double=False,globalpool=ave,dropout=0,dnodes=16,dropout=0.3,dnodes=0

Epoch 3/50
123/124 [============================>.] - ETA: 0s - loss: 1.9148 - acc: 0.7610 - auc: 0.5251
Epoch 00003: val_loss did not improve from 1.90385
124/124 [==============================] - 12s 93ms/step - loss: 1.9146 - acc: 0.7611 - auc: 0.5253 - val_loss: 1.9087 - val_acc: 0.7638 - val_auc: 0.5576
Epoch 4/50
123/124 [============================>.] - ETA: 0s - loss: 1.8986 - acc: 0.7604 - auc: 0.5454
Epoch 00004: val_loss improved from 1.90385 to 1.89913, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_only_train/batch=512,c1=128,c2=256,c3=256,filt1=5,filt2=3,filt3=7,str1=2,str2=2,str3=1,conv_double=False,globalpool=max,dropout=0.5,dnodes=32,dropout=0,dnodes=128,dropout=0.4/weights.hdf5
124/124 [==============================] - 12s 94ms/step - loss: 1.8987 - acc: 0.7604 - auc: 0.5453 - val_loss: 1.8991 - val_acc: 0.7638 - val_auc: 0.5693
Epoch 5/50
123/124 [============================>.] - ETA: 0s - loss: 1.8860 - acc: 0.7581 

KeyboardInterrupt: 

#### different structure

In [17]:
from keras import metrics
# random search for hyperparameter
ntrial = 400
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    test_setting_3 = random.choice(test_settings_3)
    test_setting_4 = random.choice(test_settings_4)    
    
        
    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2
    stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5 = test_setting_3
    dense2, dropout2 = test_setting_4
    dropout_cnn = random.choice(dropout_opts)
    
    # total conv layers of the model
    n_conv = random.choice([2,3])
    
    if n_conv==2:
        num_l3,kernel_l3,stride_l3 = 0,0,0
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
    
    if n_conv==3:
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
        
    if n_conv==4:
        num_l5,kernel_l5,stride_l5 = 0,0,0  
        
    
    n_dense = random.choice([0,1,2])
    
    if n_dense==0:
        dense1, dropout1 = 0,0
        dense2, dropout2 = 0,0
    
    if n_dense==1:
        dense2, dropout2 = 0,0
        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={},c1={},c2={},c3={},filt1={},filt2={},filt3={},str1={},str2={},str3={},conv_double={},droout_cnn={},globalpool={},dropout={},dnodes={},dropout={},dnodes={},dropout={}'.format(batch_size, num_l1, num_l2, num_l3,kernel_l1, kernel_l2, kernel_l3, stride_l1,stride_l2,stride_l3,conv_double, dropout_cnn, globalpool_opt, dropout_fc, dense1, dropout1, dense2, dropout2)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
    with strategy.scope():
        # build a model
        inp_fnn = Input(shape=(agender_train.shape[1],))
        inp_cnn = Input(shape=(x_train.shape[1], x_train.shape[2]))
    
        out_fnn = inp_fnn
        out_fnn = Activation('sigmoid') (out_fnn)
        
        out_cnn = inp_cnn
        act='relu'
        
        
        # c1 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = Dropout(dropout_cnn) (out_cnn)
        


        # c2 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = Dropout(dropout_cnn) (out_cnn)
        
        
        # c3 layer
        if n_conv>2:
            if conv_double:
                out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = Dropout(dropout_cnn) (out_cnn)
        
        
        # c4 layer
        if n_conv>3:
            if num_l3 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
            
                    


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            out_cnn = GlobalMaxPool1D() (out_cnn)
        elif globalpool_opt == 'ave':
            out_cnn = GlobalAveragePooling1D() (out_cnn)    
        out_cnn = Dropout(dropout_fc) (out_cnn)
        
        
        out = concatenate([out_fnn, out_cnn])
            
        if n_dense>=1:
            out = Dense(dense1, activation='tanh') (out)
            out = Dropout(dropout1) (out)
            
        if n_dense==2:
            out = Dense(dense2, activation='tanh') (out)
            out = Dropout(dropout2) (out)
            
        out = Dense(1, activation='sigmoid') (out)
        
        
        model = Model(inputs=[inp_cnn, inp_fnn], outputs=[out])


        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_trains, y_train_bin, sample_weight=train_w_samp, validation_data=(x_vals, y_val_bin, val_w_samp), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except Exception as e:
            print(e)
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_tests).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


random search 0/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100
124/124 [==============================] - ETA: 0s - loss: 2.0093 - acc: 0.7164 - auc_1: 0.5026
Epoch 00001: val_loss improved from inf to 1.90695, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=512,c1=256,c2=128,c3=0,filt1=3,filt2=5,filt3=0,str1=2,str2=2,str3=0,conv_double=False,droout_cnn=0.3,globalpool=max,dropout=0.5,dnodes=16,dropout=0.2,dnodes=32,dropout=0.3/weights.hdf5
124/124 [==============================] - 13s 104ms/step - loss: 2.0093 - acc: 0.7164 - auc_1: 0.5026 - val_loss: 1.9070 - val_acc: 0.7438 - val_auc_1: 0.5079
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 1.9444 - acc: 0.7513 - auc_1: 0.5008
Epoch 00002: val_loss did not improve from 1.90695
124/124 [==============================] - 10s 77ms/step - los

Epoch 1/100
62/62 [==============================] - ETA: 0s - loss: 2.3049 - acc: 0.6473 - auc: 0.4982
Epoch 00001: val_loss improved from inf to 1.95775, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=1024,c1=64,c2=32,c3=64,filt1=3,filt2=5,filt3=3,str1=1,str2=1,str3=1,conv_double=False,droout_cnn=0.2,globalpool=max,dropout=0.1,dnodes=128,dropout=0.5,dnodes=0,dropout=0/weights.hdf5
62/62 [==============================] - 9s 141ms/step - loss: 2.3049 - acc: 0.6473 - auc: 0.4982 - val_loss: 1.9577 - val_acc: 0.7438 - val_auc: 0.4416
Epoch 2/100
62/62 [==============================] - ETA: 0s - loss: 2.0679 - acc: 0.6890 - auc: 0.5064
Epoch 00002: val_loss improved from 1.95775 to 1.94406, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=1024,c1=64,c2=32,c3=64,filt1=3,filt2=5,filt3=3,str1=1,str2=1,str3=1,conv_double=False,droout_cnn=0.2,globalpool=max,dropout=0.1,dnodes=128

Epoch 3/100
124/124 [==============================] - ETA: 0s - loss: 1.8591 - acc: 0.7493 - auc: 0.5902
Epoch 00003: val_loss did not improve from 1.91267
124/124 [==============================] - 18s 143ms/step - loss: 1.8591 - acc: 0.7493 - auc: 0.5902 - val_loss: 2.7894 - val_acc: 0.2562 - val_auc: 0.4499
Epoch 4/100
124/124 [==============================] - ETA: 0s - loss: 1.8498 - acc: 0.7506 - auc: 0.5976
Epoch 00004: val_loss did not improve from 1.91267
124/124 [==============================] - 18s 143ms/step - loss: 1.8498 - acc: 0.7506 - auc: 0.5976 - val_loss: 2.3748 - val_acc: 0.3430 - val_auc: 0.4615
random search 5/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100
124/124 [==============================] - ETA: 0s - loss: 1.9975 - acc: 0.7389 - auc: 0.5162
Epoch 00001: val_loss improved from inf to 1.94170, saving model to ECG/train_val_test_pacu/1D_CNN_

Epoch 5/100
123/124 [============================>.] - ETA: 0s - loss: 1.9100 - acc: 0.7629 - auc: 0.5140
Epoch 00005: val_loss did not improve from 1.91098
124/124 [==============================] - 9s 69ms/step - loss: 1.9104 - acc: 0.7628 - auc: 0.5139 - val_loss: 1.9264 - val_acc: 0.7438 - val_auc: 0.4979
Epoch 6/100
123/124 [============================>.] - ETA: 0s - loss: 1.9061 - acc: 0.7622 - auc: 0.5185
Epoch 00006: val_loss did not improve from 1.91098
124/124 [==============================] - 9s 69ms/step - loss: 1.9061 - acc: 0.7622 - auc: 0.5183 - val_loss: 1.9301 - val_acc: 0.7438 - val_auc: 0.4957
Epoch 7/100
123/124 [============================>.] - ETA: 0s - loss: 1.9060 - acc: 0.7610 - auc: 0.5183
Epoch 00007: val_loss did not improve from 1.91098
124/124 [==============================] - 9s 69ms/step - loss: 1.9055 - acc: 0.7611 - auc: 0.5184 - val_loss: 1.9123 - val_acc: 0.7438 - val_auc: 0.4982
random search 9/400
INFO:tensorflow:Using MirroredStrategy with dev

123/124 [============================>.] - ETA: 0s - loss: 1.8592 - acc: 0.7606 - auc: 0.5869
Epoch 00008: val_loss did not improve from 1.92929
124/124 [==============================] - 9s 69ms/step - loss: 1.8592 - acc: 0.7606 - auc: 0.5871 - val_loss: 2.5239 - val_acc: 0.2562 - val_auc: 0.4345
random search 12/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Epoch 1/100
123/124 [============================>.] - ETA: 0s - loss: 2.6448 - acc: 0.6308 - auc: 0.5059
Epoch 00001: val_loss improved from inf to 1.94846, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=512,c1=64,c2=128,c3=0,filt1=5,filt2=5,filt3=0,str1=2,str2=1,str3=0,conv_double=True,droout_cnn=0.2,globalpool=max,dropout=0.1,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
124/124 [==============================] - 9s 69ms/step - loss: 2.6437 - acc: 0.6314 - auc:

...

62/62 [==============================] - ETA: 0s - loss: 1.9305 - acc: 0.7525 - auc: 0.5249
Epoch 00001: val_loss improved from inf to 2.17450, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=1024,c1=256,c2=128,c3=256,filt1=3,filt2=3,filt3=3,str1=1,str2=1,str3=1,conv_double=True,droout_cnn=0,globalpool=ave,dropout=0.2,dnodes=16,dropout=0.2,dnodes=0,dropout=0/weights.hdf5
62/62 [==============================] - 47s 761ms/step - loss: 1.9305 - acc: 0.7525 - auc: 0.5249 - val_loss: 2.1745 - val_acc: 0.7438 - val_auc: 0.5160
Epoch 2/100
62/62 [==============================] - ETA: 0s - loss: 1.8847 - acc: 0.7542 - auc: 0.5653
Epoch 00002: val_loss did not improve from 2.17450
62/62 [==============================] - 44s 703ms/step - loss: 1.8847 - acc: 0.7542 - auc: 0.5653 - val_loss: 2.2259 - val_acc: 0.7438 - val_auc: 0.5306
Epoch 3/100
62/62 [==============================] - ETA: 0s - loss: 1.8594 - acc: 0.7552 - auc: 0.5892
Ep

Epoch 1/100
123/124 [============================>.] - ETA: 0s - loss: 1.9151 - acc: 0.7576 - auc: 0.5303
Epoch 00001: val_loss improved from inf to 1.93493, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=512,c1=64,c2=128,c3=0,filt1=7,filt2=3,filt3=0,str1=1,str2=2,str3=0,conv_double=False,droout_cnn=0.3,globalpool=ave,dropout=0,dnodes=32,dropout=0.4,dnodes=0,dropout=0/weights.hdf5
124/124 [==============================] - 9s 74ms/step - loss: 1.9152 - acc: 0.7576 - auc: 0.5303 - val_loss: 1.9349 - val_acc: 0.7438 - val_auc: 0.5380
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 1.8892 - acc: 0.7624 - auc: 0.5572
Epoch 00002: val_loss improved from 1.93493 to 1.91469, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=512,c1=64,c2=128,c3=0,filt1=7,filt2=3,filt3=0,str1=1,str2=2,str3=0,conv_double=False,droout_cnn=0.3,globalpool=ave,dropout=0,dnodes=32,dr

Epoch 5/100
123/124 [============================>.] - ETA: 0s - loss: 1.9716 - acc: 0.7331 - auc: 0.5162
Epoch 00005: val_loss improved from 2.03070 to 1.96453, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=512,c1=64,c2=256,c3=64,filt1=3,filt2=3,filt3=3,str1=1,str2=2,str3=1,conv_double=False,droout_cnn=0.5,globalpool=max,dropout=0.1,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
124/124 [==============================] - 9s 70ms/step - loss: 1.9717 - acc: 0.7332 - auc: 0.5162 - val_loss: 1.9645 - val_acc: 0.7438 - val_auc: 0.4257
Epoch 6/100
123/124 [============================>.] - ETA: 0s - loss: 1.9324 - acc: 0.7573 - auc: 0.5187
Epoch 00006: val_loss improved from 1.96453 to 1.94418, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=512,c1=64,c2=256,c3=64,filt1=3,filt2=3,filt3=3,str1=1,str2=2,str3=1,conv_double=False,droout_cnn=0.5,globalpool=max,dropout=0.1,dnode

Epoch 3/100
62/62 [==============================] - ETA: 0s - loss: 1.8929 - acc: 0.7436 - auc: 0.5633
Epoch 00003: val_loss did not improve from 1.94576
62/62 [==============================] - 8s 126ms/step - loss: 1.8929 - acc: 0.7436 - auc: 0.5633 - val_loss: 1.9663 - val_acc: 0.7194 - val_auc: 0.4463
Epoch 4/100
62/62 [==============================] - ETA: 0s - loss: 1.8699 - acc: 0.7417 - auc: 0.5849
Epoch 00004: val_loss improved from 1.94576 to 1.93947, saving model to ECG/train_val_test_pacu/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_diff/batch=1024,c1=256,c2=32,c3=0,filt1=5,filt2=5,filt3=0,str1=3,str2=1,str3=0,conv_double=True,droout_cnn=0.3,globalpool=ave,dropout=0.4,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
62/62 [==============================] - 8s 128ms/step - loss: 1.8699 - acc: 0.7417 - auc: 0.5849 - val_loss: 1.9395 - val_acc: 0.7372 - val_auc: 0.5002
Epoch 5/100
62/62 [==============================] - ETA: 0s - loss: 1.8599 - acc: 0.7430 - auc:

NameError: name 'test_auc' is not defined

In [ ]:
from numba import cuda
cuda.select_device(3)
cuda.close()

## 3 class

In [ ]:
from keras import metrics
from keras.layers import LeakyReLU, ReLU
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# random search for hyperparameter
ntrial = 200
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_rmse, test_acc = [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='tanh'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(3, activation='softmax'))

        try:
            # model 학습 설정
            model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train, y_train_class, validation_data=(x_val, y_val_class), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue

    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test)

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)
    
    # auprc 
    precision, recall, _ = precision_recall_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)
    
    # acc 계산
    l_test = np.argmax(y_test_class, axis=1)
    l_pred = np.argmax(y_pred, axis=1)
    acc_val = accuracy_score(l_test, l_pred)
    test_acc.append(acc_val)
    
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

In [ ]:
from numba import cuda
cuda.select_device(2)
cuda.close()

## Regression

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 100
train_errs, val_errs = [], []
test_auc, test_rmse, test_acc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='sigmoid'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(1, activation='sigmoid'))


        # model 학습 설정
        model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["mean_absolute_error", tf.keras.metrics.AUC()])
        hist = model.fit(x_train, y_train/10, validation_split=0.1, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                            EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])


    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test).flatten()

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_auc.append(roc_auc)
    # RMSE 계산
    model_err = metrics.RootMeanSquaredError()
    model_err.update_state(y_test, y_pred)
    rmse_val = model_err.result().numpy()
    test_rmse.append(rmse_val)
    # acc 계산
    acc_val = np.mean((y_pred*10>=5)==y_test_bin)
    test_acc.append(acc_val)
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_{}_rmse{:.4f}_acc{:.2f}'.format(roc_auc, odir_f, rmse_val, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

# Results

## 1D-CNN model 1
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global maxpool)-dropout-(dense)-dropout-[output]
<br>or conv-conv-bn-maxpool

### Best Result

In [ ]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}\ninfo: {}'.format(test_auc[max_idx], random_settings[max_idx]))

### Top 10 Result

In [ ]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}   train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

## 1D-CNN model 2
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global max or ave or flatten)-dropout-(dense)-dropout-[output]

In [ ]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}  rmse:{:.4f}  acc:{:.2f}\n: {}'.format(test_auc[max_idx], test_rmse[max_idx], test_acc[max_idx], random_settings[max_idx]))

In [ ]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}  rmse:{:.4f}  acc:{:.2f},  train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]],np.array(test_rmse)[topid[9-i]], np.array(test_acc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

In [ ]:
def build_model(num_l1=256, kernel_l1 = 10, bool_flatten=False, dropout=0.2):
    #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    #with strategy.scope():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    model = Sequential()
    #num_l2=64, num_l3=64, kernel_l2=3, kernel_l3=3
    num_l2=64
    num_l3=64
    kernel_l2=3
    kernel_l3=3

    #for (num_node, kernel_size) in conv_layers:
    #    model.add(Conv1D(filters=num_node, kernel_size=kernel_size, padding='valid'))
    #    model.add(BatchNormalization())
    #    model.add(MaxPooling1D(pool_size=2))

    # Conv Layer 1
    model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 2
    model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 3
    model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # FC layer 이전의 작업
    if bool_flatten:
        model.add(Flatten())
    else:
        model.add(GlobalMaxPool1D())

    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


return model